In [1]:
# Install necessary libraries
!pip install numpy pandas scikit-learn tensorflow pyEDFlib biosppy

# Import libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, roc_auc_score, recall_score
import biosppy.signals.eda as eda
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Attention


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 87.3 MB/s eta 0:00:00


In [2]:
from scipy.signal import butter, filtfilt

def load_and_extract_eda(file_path):
    df = pd.read_csv(file_path, skiprows=3, header=None)  # Skip the first 3 rows of metadata
    eda_signal = df[0].values

    # Normalize the signal to be between 0 and 1
    eda_signal = (eda_signal - np.min(eda_signal)) / (np.max(eda_signal) - np.min(eda_signal))

    # Extract phasic and tonic components
    sampling_rate = 4.0  # E4 Empatica sampling rate is 4Hz

    # Bandpass filter parameters for phasic component
    lowcut = 0.05  # Hz
    highcut = 0.5  # Hz
    nyquist_freq = 0.5 * sampling_rate

    # Normalize frequencies
    low = lowcut / nyquist_freq
    high = highcut / nyquist_freq

    # Butterworth bandpass filter
    b, a = butter(2, [low, high], btype='bandpass')
    phasic = filtfilt(b, a, eda_signal)

    # Lowpass filter for tonic component
    lowcut_tonic = 0.05 / nyquist_freq
    b_tonic, a_tonic = butter(2, lowcut_tonic, btype='low')
    tonic = filtfilt(b_tonic, a_tonic, eda_signal)

    return phasic, tonic


# Function to load all data for a participant
def load_all_data(participant_num, num_files=10):
    phasic_list = []
    tonic_list = []
    for file_num in range(1, num_files + 1):
        file_path = f"EDA_{participant_num}_{file_num}.csv"
        phasic, tonic = load_and_extract_eda(file_path)
        phasic_list.append(phasic)
        tonic_list.append(tonic)

    return np.concatenate(phasic_list), np.concatenate(tonic_list)

# Load data for all participants
train_data_phasic, train_data_tonic = [], []
for participant_num in range(1, 5):
    phasic, tonic = load_all_data(participant_num)
    train_data_phasic.append(phasic)
    train_data_tonic.append(tonic)

train_data_phasic = np.concatenate(train_data_phasic)
train_data_tonic = np.concatenate(train_data_tonic)

# Split the data into train, validation, and test sets
train_phasic = train_data_phasic[:int(len(train_data_phasic) * 0.7)]
val_phasic = train_data_phasic[int(len(train_data_phasic) * 0.7):int(len(train_data_phasic) * 0.85)]
test_phasic = train_data_phasic[int(len(train_data_phasic) * 0.85):]

train_tonic = train_data_tonic[:int(len(train_data_tonic) * 0.7)]
val_tonic = train_data_tonic[int(len(train_data_tonic) * 0.7):int(len(train_data_tonic) * 0.85)]
test_tonic = train_data_tonic[int(len(train_data_tonic) * 0.85):]


Normalization of Data






In [3]:
scaler_phasic = MinMaxScaler(feature_range=(0, 1))
scaler_tonic = MinMaxScaler(feature_range=(0, 1))

train_phasic = scaler_phasic.fit_transform(train_phasic.reshape(-1, 1)).flatten()
val_phasic = scaler_phasic.transform(val_phasic.reshape(-1, 1)).flatten()
test_phasic = scaler_phasic.transform(test_phasic.reshape(-1, 1)).flatten()

train_tonic = scaler_tonic.fit_transform(train_tonic.reshape(-1, 1)).flatten()
val_tonic = scaler_tonic.transform(val_tonic.reshape(-1, 1)).flatten()
test_tonic = scaler_tonic.transform(test_tonic.reshape(-1, 1)).flatten()


Data Prepration

In [4]:
def create_sequences(data, time_steps=20):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

# Create sequences
time_steps = 20
X_train_phasic, y_train_phasic = create_sequences(train_phasic, time_steps)
X_val_phasic, y_val_phasic = create_sequences(val_phasic, time_steps)
X_test_phasic, y_test_phasic = create_sequences(test_phasic, time_steps)

X_train_tonic, y_train_tonic = create_sequences(train_tonic, time_steps)
X_val_tonic, y_val_tonic = create_sequences(val_tonic, time_steps)
X_test_tonic, y_test_tonic = create_sequences(test_tonic, time_steps)


Build Attention-Based LSTM Model

In [5]:
def create_attention_lstm_model(input_shape):
    input_layer = Input(shape=input_shape)
    lstm_out = LSTM(64, return_sequences=True)(input_layer)
    attention = Attention()([lstm_out, lstm_out])
    lstm_out2 = LSTM(32)(attention)
    output_layer = Dense(1)(lstm_out2)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])
    return model

input_shape = (time_steps, 1)
model_phasic = create_attention_lstm_model(input_shape)
model_tonic = create_attention_lstm_model(input_shape)


In [6]:
history_phasic = model_phasic.fit(X_train_phasic, y_train_phasic, validation_data=(X_val_phasic, y_val_phasic), epochs=10, batch_size=32)
history_tonic = model_tonic.fit(X_train_tonic, y_train_tonic, validation_data=(X_val_tonic, y_val_tonic), epochs=10, batch_size=32)


Epoch 1/10
2225/2225 ━━━━━━━━━━━━━━━━━━━━ 58s 24ms/step - loss: 0.0036 - mse: 0.0036 - val_loss: 0.0024 - val_mse: 0.0024
Epoch 2/10
2225/2225 ━━━━━━━━━━━━━━━━━━━━ 81s 24ms/step - loss: 7.8926e-04 - mse: 7.8926e-04 - val_loss: 0.0019 - val_mse: 0.0019
Epoch 3/10
2225/2225 ━━━━━━━━━━━━━━━━━━━━ 83s 25ms/step - loss: 4.7145e-04 - mse: 4.7145e-04 - val_loss: 2.1734e-04 - val_mse: 2.1734e-04
Epoch 4/10
2225/2225 ━━━━━━━━━━━━━━━━━━━━ 81s 24ms/step - loss: 9.4340e-05 - mse: 9.4340e-05 - val_loss: 9.1215e-05 - val_mse: 9.1215e-05
Epoch 5/10
2225/2225 ━━━━━━━━━━━━━━━━━━━━ 84s 25ms/step - loss: 5.8880e-05 - mse: 5.8880e-05 - val_loss: 6.9838e-05 - val_mse: 6.9838e-05
Epoch 6/10
2225/2225 ━━━━━━━━━━━━━━━━━━━━ 54s 24ms/step - loss: 3.8685e-05 - mse: 3.8685e-05 - val_loss: 6.8705e-05 - val_mse: 6.8705e-05
Epoch 7/10
2225/2225 ━━━━━━━━━━━━━━━━━━━━ 82s 24ms/step - loss: 3.3557e-05 - mse: 3.3557e-05 - val_loss: 5.8580e-05 - val_mse: 5.8580e-05
Epoch 8/10
2225/2225 ━━━━━━━━━━━━━━━━━━━━ 54s 24ms/step - 

In [7]:
def evaluate_model(model, X, y, scaler):
    y_pred = model.predict(X)
    y_pred_rescaled = scaler.inverse_transform(y_pred)
    y_rescaled = scaler.inverse_transform(y.reshape(-1, 1))

    rmse = np.sqrt(mean_squared_error(y_rescaled, y_pred_rescaled))
    accuracy = accuracy_score(np.round(y_rescaled), np.round(y_pred_rescaled))
    f1 = f1_score(np.round(y_rescaled), np.round(y_pred_rescaled), average='macro')
    sensitivity = recall_score(np.round(y_rescaled), np.round(y_pred_rescaled), average='macro')
    auroc = roc_auc_score(np.round(y_rescaled), y_pred_rescaled)

    return {"RMSE": rmse, "Accuracy": accuracy, "F1 Score": f1, "Sensitivity": sensitivity, "AUROC": auroc}

metrics_phasic = evaluate_model(model_phasic, X_test_phasic, y_test_phasic, scaler_phasic)
metrics_tonic = evaluate_model(model_tonic, X_test_tonic, y_test_tonic, scaler_tonic)
print("Phasic Component Metrics:", metrics_phasic)
print("Tonic Component Metrics:", metrics_tonic)


477/477 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step
477/477 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step
Phasic Component Metrics: {'RMSE': 0.010272080208788864, 'Accuracy': 0.9998031496062992, 'F1 Score': 0.4999507825573383, 'Sensitivity': 0.5, 'AUROC': 0.9999343702828641}
Tonic Component Metrics: {'RMSE': 0.011359482356376852, 'Accuracy': 0.9955380577427821, 'F1 Score': 0.995388559926365, 'Sensitivity': 0.9953402447163515, 'AUROC': 0.9995822024471636}


In [8]:
# Load new EDA data file
new_data_file_path = "EDA_Testing.csv"  # Replace with the actual path to your new data file

# Extract phasic and tonic components from the new data
new_phasic, new_tonic = load_and_extract_eda(new_data_file_path)

# Normalize the new data using previously fitted scalers
new_phasic_normalized = scaler_phasic.transform(new_phasic.reshape(-1, 1)).flatten()
new_tonic_normalized = scaler_tonic.transform(new_tonic.reshape(-1, 1)).flatten()

# Split the new data into 80% training input and 20% test
split_index_phasic = int(len(new_phasic_normalized) * 0.8)
split_index_tonic = int(len(new_tonic_normalized) * 0.8)

X_new_phasic = new_phasic_normalized[:split_index_phasic]
y_true_phasic = new_phasic_normalized[split_index_phasic:]

X_new_tonic = new_tonic_normalized[:split_index_tonic]
y_true_tonic = new_tonic_normalized[split_index_tonic:]


In [9]:
# Create sequences for the new phasic and tonic data
X_eval_phasic, _ = create_sequences(X_new_phasic, time_steps=len(y_true_phasic))
X_eval_tonic, _ = create_sequences(X_new_tonic, time_steps=len(y_true_tonic))


In [10]:
# Calculate the number of sequences for prediction
num_sequences = len(y_true_phasic) - time_steps + 1

# Adjust X_eval_phasic and X_eval_tonic based on the new sequence length
X_eval_phasic, _ = create_sequences(X_new_phasic, time_steps)
X_eval_tonic, _ = create_sequences(X_new_tonic, time_steps)

# Use only the necessary number of sequences that match the expected output
X_eval_phasic = X_eval_phasic[-num_sequences:]
X_eval_tonic = X_eval_tonic[-num_sequences:]


In [11]:
# Predict the remaining 20% using the trained model
y_pred_phasic = model_phasic.predict(X_eval_phasic)
y_pred_tonic = model_tonic.predict(X_eval_tonic)

# Inverse the scaling to get back to original values
y_pred_phasic = scaler_phasic.inverse_transform(y_pred_phasic)
y_true_phasic = y_true_phasic[-len(y_pred_phasic):]  # Align true values with predicted length

y_pred_tonic = scaler_tonic.inverse_transform(y_pred_tonic)
y_true_tonic = y_true_tonic[-len(y_pred_tonic):]  # Align true values with predicted length


18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [12]:
# Function to convert continuous data into binary trend
def convert_to_trend(y):
    trend = np.diff(y.flatten())  # Calculate the difference between consecutive points
    trend_binary = (trend > 0).astype(int)  # 1 for increase, 0 for decrease
    return trend_binary

# Convert y_true and y_pred into trend directions
y_true_trend_phasic = convert_to_trend(y_true_phasic)
y_pred_trend_phasic = convert_to_trend(y_pred_phasic)

y_true_trend_tonic = convert_to_trend(y_true_tonic)
y_pred_trend_tonic = convert_to_trend(y_pred_tonic)



In [13]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score

# Function to evaluate trend-based metrics
def evaluate_trend(y_true_trend, y_pred_trend):
    accuracy = accuracy_score(y_true_trend, y_pred_trend)
    f1 = f1_score(y_true_trend, y_pred_trend, average='macro')
    sensitivity = recall_score(y_true_trend, y_pred_trend, average='macro')
    auroc = roc_auc_score(y_true_trend, y_pred_trend)

    return {"Accuracy": accuracy, "F1 Score": f1, "Sensitivity": sensitivity, "AUROC": auroc}

# Evaluate phasic component trend
metrics_new_phasic_trend = evaluate_trend(y_true_trend_phasic, y_pred_trend_phasic)
print("New Phasic Component Trend Metrics:", metrics_new_phasic_trend)

# Evaluate tonic component trend
metrics_new_tonic_trend = evaluate_trend(y_true_trend_tonic, y_pred_trend_tonic)
print("New Tonic Component Trend Metrics:", metrics_new_tonic_trend)


New Phasic Component Trend Metrics: {'Accuracy': 0.5080789946140036, 'F1 Score': 0.5080012894906512, 'Sensitivity': 0.509886275523391, 'AUROC': 0.509886275523391}
New Tonic Component Trend Metrics: {'Accuracy': 0.718132854578097, 'F1 Score': 0.41797283176593525, 'Sensitivity': 0.3853564547206166, 'AUROC': 0.3853564547206166}
